# Module 3: MUVERA

In [1]:
from fastembed import LateInteractionMultimodalEmbedding
from fastembed.postprocess import Muvera

# Load ColModernVBERT model
model = LateInteractionMultimodalEmbedding(model_name="Qdrant/colmodernvbert")

# Wrap with MUVERA processor
muvera = Muvera.from_multivector_model(
    model=model,
    k_sim=6,        # 2^6 = 64 similarity buckets
    dim_proj=32,    # Projection dimensionality
    r_reps=20       # Random projection repetitions
)

In [3]:
from qdrant_client import QdrantClient, models

# Create collection with both vector types
client = QdrantClient("http://localhost:6333")
client.create_collection(
    collection_name="documents-muvera",
    vectors_config={
        "muvera": models.VectorParams(
            size=muvera.embedding_size,
            distance=models.Distance.COSINE
        ),
        "colmodernvbert": models.VectorParams(
            size=model.embedding_size,
            distance=models.Distance.COSINE,
            multivector_config=models.MultiVectorConfig(
                comparator=models.MultiVectorComparator.MAX_SIM
            )
        )
    }
)

True

In [4]:
# Embed documents
image_path = "images/financial-report.png"
doc_multivec = list(model.embed_image([image_path]))[0]
doc_muvera = muvera.process_document(doc_multivec)

In [5]:
# Upload both representations
client.upsert(
    collection_name="documents-muvera",
    points=[
        models.PointStruct(
            id=0,
            payload={"source": image_path},
            vector={"muvera": doc_muvera, "colmodernvbert": doc_multivec}
        )
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [8]:
# Encode query in both formats
query = "quarterly revenue growth"
query_multivec = list(model.embed_text(query))[0]
query_muvera = muvera.process_query(query_multivec)

In [9]:
# Two-stage retrieval with Qdrant's prefetch
results = client.query_points(
    collection_name="documents-muvera",
    prefetch=models.Prefetch(
        query=query_muvera,
        using="muvera",
        limit=100,  # Stage 1: Fast MUVERA retrieval
    ),
    query=query_multivec,
    using="colmodernvbert",  # Stage 2: Precise MaxSim reranking
    limit=10,
    with_payload=True
)

In [10]:
results

QueryResponse(points=[ScoredPoint(id=0, version=0, score=8.957454, payload={'source': 'images/financial-report.png'}, vector=None, shard_key=None, order_value=None)])